In [85]:
import pandas as pd

In [86]:
df = pd.read_parquet('../data/news_feed_flattened.parquet')

In [ ]:
# first entry
df.head(5)

In [ ]:
# last entry
df.tail(5)

In [89]:
# number of entries
df.link.nunique()

49

* Check the retrieval code and work on the same for some query from the Alexey's code

# Retrieval

Link refered to for Retrieval: https://github.com/alexeygrigorev/build-your-own-search-engine/blob/main/notebook.ipynb

## Individual news

In [ ]:
ind_news_df = df[df.type == 'individual'].head()
ind_news_df.head(3)

## ind_news_dfEmbeding of context using Tfidfvectorizer

In [ ]:
print(ind_news_df.text.values[:2])

In [92]:
news_texts = ind_news_df.text.values

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [94]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(news_texts)

In [95]:
names = vectorizer.get_feature_names_out()

In [ ]:
df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)

In [97]:
query = 'What are the signifcant milestones for Tesla from 2023-07-17 to 2023-07-24 ? ' 
# what should be the query here to search for relevant text from the dataframe?

In [ ]:
q = vectorizer.transform([query])
q.toarray()

In [ ]:
query_dict = dict(zip(names, q.toarray()[0]))
query_dict

In [ ]:
doc_dict = dict(zip(names, X.toarray()[1]))
doc_dict

In [101]:
df_qd = pd.DataFrame([query_dict, doc_dict], index=['query', 'doc']).T

In [102]:
(df_qd['query'] * df_qd['doc']).sum()

np.float64(0.0)

In [103]:
X.dot(q.T).toarray()

array([[0.27498098],
       [0.        ],
       [0.        ],
       [0.10141555],
       [0.        ]])

In [104]:
from sklearn.metrics.pairwise import cosine_similarity

In [105]:
cosine_similarity(X, q)

array([[0.27498098],
       [0.        ],
       [0.        ],
       [0.10141555],
       [0.        ]])

In [106]:
df.columns

Index(['type', 'start_date', 'end_date', 'ticker', 'count', 'growth', 'text',
       'link', 'model'],
      dtype='object')

In [107]:
df.growth.isnull().sum()

np.int64(119)

In [113]:
fields = ['text', 'end_date','start_date', 'end_date']
# growth should also be present; because of some null we are skipping it
transformers = {}
matrices = {}

for field in fields:
    print(f'processing for field: {field}')
    vectorizer = TfidfVectorizer(stop_words='english', min_df=3)
    X = vectorizer.fit_transform(df[field])

    transformers[field] = vectorizer
    matrices[field] = X

processing for field: text
processing for field: end_date
processing for field: start_date
processing for field: end_date


In [114]:
transformers['text'].get_feature_names_out()

array(['00', '000', '05', ..., 'zero', 'zone', 'zuckerberg'], dtype=object)

In [115]:
matrices['text']

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 40618 stored elements and shape (635, 2671)>

In [116]:
q = transformers['text'].transform([query])

In [117]:
q 

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (1, 2671)>

In [118]:
score = cosine_similarity(matrices['text'], q).flatten()

### Search with boosting

In [119]:
boost = {'text': 3.0, 'start_date':2, 'end_date':1 } 
# text if related  info is present
# start_date if query gives the from date. in my opinion this should be hard coded while fetchin the data itself
# end_date to cut off the info at a timeframe so the query cna be answered properly

In [120]:
import numpy as np

In [121]:
score = np.zeros(len(df))

In [122]:
for f in fields:
    b = boost.get(f, 1.0)
    q = transformers[f].transform([query])
    s = cosine_similarity(matrices[f], q).flatten()
    score = score + b * s

In [126]:
query

'What are the signifcant milestones for Tesla from 2023-07-17 to 2023-07-24 ? '

In [125]:
# results -> shows relevant docs for the query based on the given context
idx = np.argsort(-score)[:1] # the filtering should be stringent because we don't wnat any other stock to pop up if we are searching for tesla
results = df.iloc[idx]
results.to_dict(orient='records')

[{'type': 'individual',
  'start_date': '2023-07-17',
  'end_date': '2023-07-24',
  'ticker': 'TSLA',
  'count': 42,
  'growth': '2.43%',
  'text': 'Billionaire investor Chamath Palihapitiya believes that Tesla is experiencing its "iPhone moment" and is on the verge of a major breakthrough. Meanwhile, investor David Trainer argues that Tesla is overvalued by over 1,000% due to its "disconnected" fundamentals. Tesla\'s second-quarter earnings report showed a rise in sales but also highlighted concerns about profitability and margins. Tesla\'s stock price dropped around 9% in response to the report. Despite mixed opinions, some analysts remain bullish on Tesla\'s future growth potential.',
  'link': 'https://pythoninvest.com/tpost/yk09rupzv1-week-17-24-july-2023',
  'model': None}]

# Improvements

* using sentence transformers
    * this will help you understand the wuery and give to better results.